<a href="https://colab.research.google.com/github/cifkao/ss-vq-vae/blob/main/experiments/colab_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Timbre transfer demo

Copyright 2020 InterDigital R&D and Télécom Paris.  
Author: Ondřej Cífka

## Download the model

In [1]:
logdir = 'model'
logdir_original = 'model-original'

In [2]:
!wget https://adasp.telecom-paris.fr/rc-ext/demos_companion-pages/vqvae_examples/ssvqvae_model_state.pt -O $logdir_original/model_state.pt

--2023-08-07 17:23:37--  https://adasp.telecom-paris.fr/rc-ext/demos_companion-pages/vqvae_examples/ssvqvae_model_state.pt
Resolving adasp.telecom-paris.fr (adasp.telecom-paris.fr)... 137.194.22.227, 2a04:8ec0:0:a::89c2:16e3
Connecting to adasp.telecom-paris.fr (adasp.telecom-paris.fr)|137.194.22.227|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 222788899 (212M)
Saving to: ‘model-original/model_state.pt’

model-original/mode 100%[===================>] 212.47M  27.0MB/s    in 7.4s    

2023-08-07 17:23:45 (28.8 MB/s) - ‘model-original/model_state.pt’ saved [222788899/222788899]



## Load the model

In [1]:
import sys
# sys.path.append('/home/wk406185/PSP/ss-vq-vae')
# sys.path.append('/home/wk406185/PSP/ss-vq-vae/src')
# sys.path.append('/home/wk406185/PSP/ss-vq-vae/src/ss_vq_vae')

In [2]:
import os

import confugue
# from ddsp.colab import colab_utils
import librosa
import torch

from ss_vq_vae.models.vqvae_oneshot import Experiment, ExperimentZeroShot

2024-02-21 14:01:51.053621: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-21 14:01:51.888353: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Load the zero-shot model

In [15]:
logdir = '/mnt/vdb/model-leaky-relu-frozen-style-pretraining-29-10-2023/'
cfg = confugue.Configuration.from_yaml_file(os.path.join(logdir, 'config-zero-shot.yaml'))
exp = cfg.configure(ExperimentZeroShot, logdir=logdir, device='cuda', pretrained_style_encoder=False, continue_training=False, frozen_style_encoder=False, continue_step=0)
exp.model.load_state_dict(torch.load(os.path.join(logdir, 'model_state.pt'), map_location=exp.device))
exp.model.train(False)

/home/user/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/user/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ModelZeroShotUnfrozenStyle(
  (content_encoder): Sequential(
    (0): Conv1d(1025, 1024, kernel_size=(4,), stride=(2,), padding=(2,))
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1)
    (3): Conv1d(1024, 1024, kernel_size=(4,), stride=(2,), padding=(2,))
    (4): ResidualWrapper(
      (module): Sequential(
        (0): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): LeakyReLU(negative_slope=0.1)
        (2): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,))
      )
    )
    (5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (vq): VQEmbedding(
    (embedding): Embedding(2048, 1024)
  )
  (style_encoder): COLA(
    (encoder): EfficientNetEmbedding(
      (backend): EfficientNet(
        (features): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), paddin

In [4]:
INPUT_ROOT = 'https://adasp.telecom-paris.fr/rc-ext/demos_companion-pages/vqvae_examples/'
INPUT_URLS = {
    'Electric Guitar': INPUT_ROOT + 'real/content/UnicornRodeo_Maybe_UnicornRodeo_Maybe_Full_25_ElecGtr2CloseMic3.0148.mp3',
    'Electric Organ': INPUT_ROOT + 'real/style/AllenStone_Naturally_Allen%20Stone_Naturally_Keys-Organ-Active%20DI.0253.mp3',
    'Jazz Piano': INPUT_ROOT + 'real/style/MaurizioPagnuttiSextet_AllTheGinIsGone_MaurizioPagnuttiSextet_AllTheGinIsGone_Full_12_PianoMics1.08.mp3',
    'Synth': INPUT_ROOT + 'real/content/Skelpolu_TogetherAlone_Skelpolu_TogetherAlone_Full_13_Synth.0190.mp3'
}

## Choose or record inputs

In [5]:
import IPython.display as ipd
import librosa

#@title Content input
content_input = 'Jazz Piano'  #@param ["Record", "Electric Guitar", "Electric Organ", "Jazz Piano", "Synth"]
record_seconds = 8 #@param {type:"number"}

if content_input == 'Record':
    a_content = colab_utils.record(seconds=record_seconds, sample_rate=exp.sr, normalize_db=0.1)
else:
    !wget {INPUT_URLS[content_input]} -O content_input.mp3
    a_content, _ = librosa.load('content_input.mp3', sr=exp.sr)
# colab_utils.play(a_content, sample_rate=exp.sr)
ipd.Audio(a_content, rate=exp.sr)

--2023-11-12 19:36:16--  https://adasp.telecom-paris.fr/rc-ext/demos_companion-pages/vqvae_examples/real/style/MaurizioPagnuttiSextet_AllTheGinIsGone_MaurizioPagnuttiSextet_AllTheGinIsGone_Full_12_PianoMics1.08.mp3
Resolving adasp.telecom-paris.fr (adasp.telecom-paris.fr)... 137.194.22.227, 2a04:8ec0:0:a::89c2:16e3
Connecting to adasp.telecom-paris.fr (adasp.telecom-paris.fr)|137.194.22.227|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162004 (158K) [audio/mpeg]
Saving to: ‘content_input.mp3’

content_input.mp3   100%[===================>] 158.21K  --.-KB/s    in 0.09s   

2023-11-12 19:36:16 (1.65 MB/s) - ‘content_input.mp3’ saved [162004/162004]



NameError: name 'exp' is not defined

In [23]:
#@title Style input
style_input = 'Electric Guitar'  #@param ["Record", "Electric Guitar", "Electric Organ", "Jazz Piano", "Synth"]
record_seconds = 8 #@param {type:"number"}

if style_input == 'Record':
    a_style = colab_utils.record(seconds=record_seconds, sample_rate=exp.sr, normalize_db=0.1)
else:
    !wget {INPUT_URLS[style_input]} -O style_input.mp3
    a_style, _ = librosa.load('style_input.mp3', sr=exp.sr)
# colab_utils.play(a_style, sample_rate=16000)
ipd.Audio(a_style, rate=exp.sr) 

--2023-11-05 19:22:27--  https://adasp.telecom-paris.fr/rc-ext/demos_companion-pages/vqvae_examples/real/content/UnicornRodeo_Maybe_UnicornRodeo_Maybe_Full_25_ElecGtr2CloseMic3.0148.mp3
Resolving adasp.telecom-paris.fr (adasp.telecom-paris.fr)... 137.194.22.227, 2a04:8ec0:0:a::89c2:16e3
Connecting to adasp.telecom-paris.fr (adasp.telecom-paris.fr)|137.194.22.227|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162004 (158K) [audio/mpeg]
Saving to: ‘style_input.mp3’

style_input.mp3     100%[===================>] 158.21K  --.-KB/s    in 0.09s   

2023-11-05 19:22:27 (1.73 MB/s) - ‘style_input.mp3’ saved [162004/162004]



## Run the model

In [24]:
s_content = torch.as_tensor(exp.preprocess(a_content), device=exp.device)[None, :]
s_style = torch.as_tensor(exp.melspectrogram_preprocess(a_style, 16_000), device=exp.device)[None, :]
l_content, l_style = (torch.as_tensor([x.shape[2]], device=exp.device) for x in [s_content, s_style])


In [25]:
s_style.shape

torch.Size([1, 64, 801])

In [6]:
import numpy as np

# with torch.no_grad():
#     s_output = exp.model(input_c=s_content, input_s=s_style,
#                          length_c=l_content, length_s=l_style, output_c=None, output_l=None)
with torch.no_grad():
    s_output = exp.model(input_c=s_content, input_s=s_style,
                         length_c=l_content, length_s=l_style, output_c=None, output_l=None)
# s_output
x = s_output[0].cpu().numpy()
print(x.shape)
a_output = exp.postprocess(s_output[0].cpu().numpy())
# a_output /= a_output.max()
# colab_utils.play(a_output, sample_rate=16000)
ipd.Audio(a_output, rate=16_000)
# np.isnan(a_output).any()

NameError: name 'exp' is not defined

# Calculate LSD on random_audios_data

## Load data

In [3]:
import pandas as pd

In [4]:
# ground_audios_df = pd.read_csv("../data/random_audios/generated_files.csv")
ground_audios_df = pd.read_csv("/mnt/vdb/test_set.csv", header=None, delimiter='\t')
ground_audios_df.head()

,0,1,2
0,/mnt/vdb/random_audios_patch_16k//data_169_035...,/mnt/vdb/random_audios_patch_16k//data_984_014...,/mnt/vdb/random_audios_patch_16k//data_169_014...
1,/mnt/vdb/random_audios_patch_16k//data_745_014...,/mnt/vdb/random_audios_patch_16k//data_908_029...,/mnt/vdb/random_audios_patch_16k//data_745_029...
2,/mnt/vdb/random_audios_patch_16k//data_454_025...,/mnt/vdb/random_audios_patch_16k//data_149_057...,/mnt/vdb/random_audios_patch_16k//data_454_057...
3,/mnt/vdb/random_audios_patch_16k//data_672_060...,/mnt/vdb/random_audios_patch_16k//data_497_090...,/mnt/vdb/random_audios_patch_16k//data_672_090...
4,/mnt/vdb/random_audios_patch_16k//data_457_034...,/mnt/vdb/random_audios_patch_16k//data_588_019...,/mnt/vdb/random_audios_patch_16k//data_457_019...


In [5]:
with open("outputs/random_audios/vqvae_list") as f:
    generated_audios_list = f.read().splitlines()
generated_audios_list[:10]

['outputs/random_audios/vqvae000.wav',
 'outputs/random_audios/vqvae001.wav',
 'outputs/random_audios/vqvae002.wav',
 'outputs/random_audios/vqvae003.wav',
 'outputs/random_audios/vqvae004.wav',
 'outputs/random_audios/vqvae005.wav',
 'outputs/random_audios/vqvae006.wav',
 'outputs/random_audios/vqvae007.wav',
 'outputs/random_audios/vqvae008.wav',
 'outputs/random_audios/vqvae009.wav']

In [12]:
import IPython.display as ipd

HOME_ROOT = "/home/user/random_audios_grouped/random_audios_test_16k"
rate = 16_000

content_path, style_path, reference_path = ground_audios_df.iloc[0]

ipd.Audio(f"{HOME_ROOT}/{content_path}", rate=rate)

ValueError: could not convert string to float: '/home/user/random_audios_grouped/random_audios_test_16k//mnt/vdb/random_audios_patch_16k//data_586_005.wav'

In [25]:
ipd.Audio(f"{HOME_ROOT}/{style_path}", rate=rate)

In [26]:
ipd.Audio(f"{HOME_ROOT}/{reference_path}", rate=rate)

In [27]:
GENERATED_ROOT = "/home/user/ss-vq-vae/experiments/outputs/random_audios/"
generated_sample = generated_audios_list[0]

ipd.Audio(f"{GENERATED_ROOT}/{generated_sample}", rate=rate)

In [28]:
assert len(generated_audios_list) == len(ground_audios_df)

## Calculate LSD

In [9]:
import os
import librosa
import numpy as np

def normalize_power(audio):
    return audio / (np.sqrt(np.mean(audio ** 2)) + np.finfo(audio.dtype).eps)

def eval_example_target(output, reference, rate=16_000):
    def spec(audio):
        audio = normalize_power(audio)
        s = librosa.feature.melspectrogram(y=audio, sr=rate)
        return librosa.power_to_db(s)
    
    s_out, s_ref = spec(output), spec(reference)
    lsd = np.mean(np.sqrt(np.mean((s_out - s_ref) ** 2, axis=1)))

    return {'lsd': lsd}

## Calculate pitch jaccard

In [10]:
from ss_vq_vae.util import compare_chroma
import essentia.standard as estd
import pretty_midi

def get_pitches(audio, rate=16_000):
    input_sr, sr = rate, 8000  # Need to resample because of EqualLoudness
    audio = estd.Resample(inputSampleRate=input_sr, outputSampleRate=sr)(audio)
    audio = estd.EqualLoudness(sampleRate=sr)(audio)
    rng = np.random.default_rng(seed=(audio > 0).sum())
    audio = rng.normal(loc=audio, scale=1e-4).astype(audio.dtype)  # To prevent Melodia from crashing
    pitches = estd.MultiPitchMelodia(sampleRate=sr)(audio)
    pitches = [[pretty_midi.utilities.hz_to_note_number(p) for p in pl if not np.isclose(0, p)]
               for pl in pitches]
    pitches = [[int(p + 0.5) for p in pl] for pl in pitches]
    return pitches

def eval_example_content(output, reference):
    pitches_output, pitches_reference = get_pitches(output), get_pitches(reference)
    assert len(pitches_output) == len(pitches_reference)
    jaccard = []
    for pl_output, pl_reference in zip(pitches_output, pitches_reference):
        matches = len(set(pl_output) & set(pl_reference))
        total = len(set(pl_output) | set(pl_reference))
        if total == 0:
            jaccard.append(0)
        else:
            jaccard.append(1 - matches / total)
    jaccard = np.mean(jaccard)
    return {'pitch_jaccard': jaccard}

def eval_example_content_chroma(output, reference):
    # Calculate MSE between two chromagrams
    pitch_chroma_mse = compare_chroma(output, reference)
    return {'pitch_chroma_mse': pitch_chroma_mse}

[   INFO   ] MusicExtractorSVM: no classifier models were configured by default


## Function `get_stats`

In [11]:
from tqdm import tqdm

def get_stats(ground_audios_df, generated_custom_audios_list, rate=16_000, HOME_ROOT="", GENERATED_CUSTOM_ROOT=""):
    lsds = []
    content_pitches = []
    content_chromas = []
    for (_, (_, _, ground_path)), generated_path in tqdm(zip(ground_audios_df.iterrows(), generated_custom_audios_list), total=len(ground_audios_df)):
        ground_path = os.path.join(HOME_ROOT, ground_path)
        generated_path = os.path.join(GENERATED_CUSTOM_ROOT, generated_path)

        ground_audio, _ = librosa.load(ground_path, sr=rate, mono=True)
        generated_audio, _ = librosa.load(generated_path, sr=rate, mono=True)
        min_length = min(len(generated_audio), len(ground_audio))
        generated_audio = generated_audio[:min_length]
        ground_audio = ground_audio[:min_length]

        lsds.append(eval_example_target(generated_audio, ground_audio)['lsd'])
        content_pitches.append(eval_example_content(generated_audio, ground_audio)['pitch_jaccard'])
        content_chromas.append(eval_example_content_chroma(generated_audio, ground_audio)['pitch_chroma_mse'])

    return lsds, content_pitches, content_chromas

def print_stats(model_name, ground_audios_df, generated_audios_file, generated_audios_root):
    with open(generated_audios_file, 'r') as f:
        generated_audios_list = f.read().splitlines()

    lsds, content_pitches, content_chromas = get_stats(
        ground_audios_df, 
        generated_audios_list,
        GENERATED_CUSTOM_ROOT=generated_audios_root
    )
    print_message = '\n'.join([
        f"{model_name} LSD: {sum(lsds) / len(lsds)}",
        f"{model_name} Pitch jaccard: {sum(content_pitches) / len(content_pitches)}",
        f"{model_name} Pitch chroma MSE: {sum(content_chromas) / len(content_chromas)}",
    ])
    print(print_message)
    return model_name, lsds, content_pitches, content_chromas

In [13]:
from multiprocessing import Pool
from tqdm import tqdm

fn_args = [
    # ('model-leaky-relu-finetuned-style-pretraining-29-10-2023', ground_audios_df, 'outputs/model-leaky-relu-finetuned-style-pretraining-29-10-2023/vqvae_list', 'outputs/model-leaky-relu-finetuned-style-pretraining-29-10-2023/'),
    # ('model-leaky-relu-frozen-style-pretraining-29-10-2023', ground_audios_df, 'outputs/model-leaky-relu-frozen-style-pretraining-29-10-2023/vqvae_list', 'outputs/model-leaky-relu-frozen-style-pretraining-29-10-2023/'),
    # ('model-leaky-relu-no-style-pretraining-28-10-2023', ground_audios_df, 'outputs/model-leaky-relu-no-style-pretraining-28-10-2023/vqvae_list', 'outputs/model-leaky-relu-no-style-pretraining-28-10-2023/'),
    # ('model-leaky-original-08-10-2023', ground_audios_df, 'outputs/model-original-08-10-2023/vqvae_list', 'outputs/model-original-08-10-2023/'),
    # ('model-leaky-original-pretrained-frozen-25-10-2023', ground_audios_df, 'outputs/model-original-pretrained-frozen-25-10-2023/vqvae_list', 'outputs/model-original-pretrained-frozen-25-10-2023/'),
    # ('model-leaky-original-pretrained-unfrozen-24-10-2023', ground_audios_df, 'outputs/model-original-pretrained-unfrozen-24-10-2023/vqvae_list', 'outputs/model-original-pretrained-unfrozen-24-10-2023/'),
    ('model-leaky-relu-finetuned-style-pretraining-15-11-2023', ground_audios_df, 'outputs/model-leaky-relu-finetuned-style-pretraining-15-11-2023/vqvae_list', 'outputs/model-leaky-relu-finetuned-style-pretraining-15-11-2023/'),
    ('model-leaky-relu-frozen-style-pretraining-15-11-2023', ground_audios_df, 'outputs/model-leaky-relu-frozen-style-pretraining-15-11-2023/vqvae_list', 'outputs/model-leaky-relu-frozen-style-pretraining-15-11-2023/'),
    ('model-leaky-relu-no-style-pretraining-13-11-2023', ground_audios_df, 'outputs/model-leaky-relu-no-style-pretraining-13-11-2023/vqvae_list', 'outputs/model-leaky-relu-no-style-pretraining-13-11-2023/'),
    ('model-original-finetuned-style-pretraining-22-11-2023', ground_audios_df, 'outputs/model-original-finetuned-style-pretraining-22-11-2023/vqvae_list', 'outputs/model-original-finetuned-style-pretraining-22-11-2023/'),
    ('model-original-frozen-style-pretraining-21-11-2023', ground_audios_df, 'outputs/model-original-frozen-style-pretraining-21-11-2023/vqvae_list', 'outputs/model-original-frozen-style-pretraining-21-11-2023/'),
    ('model-original-no-style-pretraining-19-11-2023', ground_audios_df, 'outputs/model-original-no-style-pretraining-19-11-2023/vqvae_list', 'outputs/model-original-no-style-pretraining-19-11-2023/')
]

def print_stats_wrapper(args_tuple):
    return print_stats(*args_tuple)

# with Pool(processes=4) as pool:
#     results = [result for result in tqdm(pool.imap(print_stats_wrapper, fn_args))]
    
# results = list(results)
results = [print_stats_wrapper(arg_tuple) for arg_tuple in fn_args]

100%|██████████| 1661/1661 [13:03<00:00,  2.12it/s]


model-leaky-relu-finetuned-style-pretraining-15-11-2023 LSD: 15.206020320201048
model-leaky-relu-finetuned-style-pretraining-15-11-2023 Pitch jaccard: 0.6561444675652507
model-leaky-relu-finetuned-style-pretraining-15-11-2023 Pitch chroma MSE: 0.08832172993043508


100%|██████████| 1661/1661 [12:57<00:00,  2.14it/s]


model-leaky-relu-frozen-style-pretraining-15-11-2023 LSD: 12.782366768126744
model-leaky-relu-frozen-style-pretraining-15-11-2023 Pitch jaccard: 0.5668364671873812
model-leaky-relu-frozen-style-pretraining-15-11-2023 Pitch chroma MSE: 0.08874204469208866


100%|██████████| 1661/1661 [12:40<00:00,  2.18it/s]


...

model-original-finetuned-style-pretraining-22-11-2023 LSD: 7.8197573584867195
model-original-finetuned-style-pretraining-22-11-2023 Pitch jaccard: 0.41978896128182125
model-original-finetuned-style-pretraining-22-11-2023 Pitch chroma MSE: 0.061258230351633795


100%|██████████| 1661/1661 [12:45<00:00,  2.17it/s]


model-original-frozen-style-pretraining-21-11-2023 LSD: 12.051983500876245
model-original-frozen-style-pretraining-21-11-2023 Pitch jaccard: 0.7092122133535608
model-original-frozen-style-pretraining-21-11-2023 Pitch chroma MSE: 0.10912459880149486


100%|██████████| 1661/1661 [12:37<00:00,  2.19it/s]

model-original-no-style-pretraining-19-11-2023 LSD: 10.15087476284801
model-original-no-style-pretraining-19-11-2023 Pitch jaccard: 0.5206190455571835
model-original-no-style-pretraining-19-11-2023 Pitch chroma MSE: 0.07881765983312479


### Calculate the original model's LSD (from the author's of the original paper)

In [30]:
import os

lsds = []
for (_, (_, _, ground_path)), generated_path in zip(ground_audios_df.iterrows(), generated_audios_list):
    ground_path = os.path.join(HOME_ROOT, ground_path)
    generated_path = os.path.join(GENERATED_ROOT, generated_path)
    
    ground_audio, _ = librosa.load(ground_path, sr=rate, mono=True)
    generated_audio, _ = librosa.load(generated_path, sr=rate, mono=True)
    min_length = min(len(generated_audio), len(ground_audio))
    generated_audio = generated_audio[:min_length]
    ground_audio = ground_audio[:min_length]

    lsds.append(eval_example_target(generated_audio, ground_audio)['lsd'])

print(sum(lsds) / len(lsds))

90.93506548242661


### Calculate COLA model with finetuned style encoder (`model-leaky-relu-finetuned-style-pretraining-29-10-2023`)

In [20]:
import os
from tqdm import tqdm

rate = 16_000
HOME_ROOT = ""
GENERATED_CUSTOM_ROOT = "/home/user/ss-vq-vae/experiments/outputs/model-leaky-relu-finetuned-style-pretraining-29-10-2023/"
with open("outputs/model-leaky-relu-finetuned-style-pretraining-29-10-2023/vqvae_list") as f:
    generated_custom_audios_list = f.read().splitlines()

lsds = []
content_pitches = []
content_chromas = []
for (_, (_, _, ground_path)), generated_path in tqdm(zip(ground_audios_df.iterrows(), generated_custom_audios_list)):
    ground_path = os.path.join(HOME_ROOT, ground_path)
    generated_path = os.path.join(GENERATED_CUSTOM_ROOT, generated_path)
    
    ground_audio, _ = librosa.load(ground_path, sr=rate, mono=True)
    generated_audio, _ = librosa.load(generated_path, sr=rate, mono=True)
    min_length = min(len(generated_audio), len(ground_audio))
    generated_audio = generated_audio[:min_length]
    ground_audio = ground_audio[:min_length]

    lsds.append(eval_example_target(generated_audio, ground_audio)['lsd'])
    content_pitches.append(eval_example_content(generated_audio, ground_audio)['pitch_jaccard'])
    content_chromas.append(eval_example_content_chroma(generated_audio, ground_audio)['pitch_chroma_mse'])

print(f"LSD: {sum(lsds) / len(lsds)}")
print(f"Pitch jaccard: {sum(content_pitches) / len(content_pitches)}")
print(f"Pitch chroma MSE: {sum(content_chromas) / len(content_chromas)}")

751it [05:35,  2.24it/s]


KeyboardInterrupt: 

### Calculate COLA model with frozen pretrained style encoder (`model-leaky-relu-frozen-style-pretraining-29-10-2023`)

In [ ]:
import os

rate = 16_000
HOME_ROOT = ""
GENERATED_CUSTOM_ROOT = "/home/user/ss-vq-vae/experiments/outputs/model-leaky-relu-frozen-style-pretraining-29-10-2023/"
with open("outputs/model-leaky-relu-frozen-style-pretraining-29-10-2023/vqvae_list") as f:
    generated_custom_audios_list = f.read().splitlines()

lsds = []
content_pitches = []
content_chromas = []
for (_, (_, _, ground_path)), generated_path in tqdm(zip(ground_audios_df.iterrows(), generated_custom_audios_list)):
    ground_path = os.path.join(HOME_ROOT, ground_path)
    generated_path = os.path.join(GENERATED_CUSTOM_ROOT, generated_path)
    
    ground_audio, _ = librosa.load(ground_path, sr=rate, mono=True)
    generated_audio, _ = librosa.load(generated_path, sr=rate, mono=True)
    min_length = min(len(generated_audio), len(ground_audio))
    generated_audio = generated_audio[:min_length]
    ground_audio = ground_audio[:min_length]

    lsds.append(eval_example_target(generated_audio, ground_audio)['lsd'])
    content_pitches.append(eval_example_content(generated_audio, ground_audio)['pitch_jaccard'])
    content_chromas.append(eval_example_content_chroma(generated_audio, ground_audio)['pitch_chroma_mse'])

print(f"LSD: {sum(lsds) / len(lsds)}")
print(f"Pitch jaccard: {sum(content_pitches) / len(content_pitches)}")
print(f"Pitch chroma MSE: {sum(content_chromas) / len(content_chromas)}")

### Calculate COLA model with no pretrained style encoder (`model-leaky-relu-no-style-pretraining-28-10-2023`)

In [ ]:
import os

rate = 16_000
HOME_ROOT = ""
GENERATED_CUSTOM_ROOT = "/home/user/ss-vq-vae/experiments/outputs/model-leaky-relu-no-style-pretraining-28-10-2023/"
with open("outputs/model-leaky-relu-no-style-pretraining-28-10-2023/vqvae_list") as f:
    generated_custom_audios_list = f.read().splitlines()

lsds = []
content_pitches = []
content_chromas = []
for (_, (_, _, ground_path)), generated_path in zip(ground_audios_df.iterrows(), generated_custom_audios_list):
    ground_path = os.path.join(HOME_ROOT, ground_path)
    generated_path = os.path.join(GENERATED_CUSTOM_ROOT, generated_path)
    
    ground_audio, _ = librosa.load(ground_path, sr=rate, mono=True)
    generated_audio, _ = librosa.load(generated_path, sr=rate, mono=True)
    min_length = min(len(generated_audio), len(ground_audio))
    generated_audio = generated_audio[:min_length]
    ground_audio = ground_audio[:min_length]

    lsds.append(eval_example_target(generated_audio, ground_audio)['lsd'])
    content_pitches.append(eval_example_content(generated, ground_audio)['pitch_jaccard'])
    content_chromas.append(eval_example_content_chroma(generated_audio, ground_audio)['pitch_chroma_mse'])

print(f"LSD: {sum(lsds) / len(lsds)}")
print(f"Pitch jaccard: {sum(content_pitches) / len(content_pitches)}")
print(f"Pitch chroma MSE: {sum(content_chromas) / len(content_chromas)}")

### Calculate original model with no pretrained style encoder (`model-original-08-10-2023`)

In [ ]:
import os

rate = 16_000
HOME_ROOT = ""
GENERATED_CUSTOM_ROOT = "/home/user/ss-vq-vae/experiments/outputs/model-original-08-10-2023/"
with open("outputs/model-original-08-10-2023/vqvae_list") as f:
    generated_custom_audios_list = f.read().splitlines()

lsds = []
content_pitches = []
content_chromas = []
for (_, (_, _, ground_path)), generated_path in tqdm(zip(ground_audios_df.iterrows(), generated_custom_audios_list)):
    ground_path = os.path.join(HOME_ROOT, ground_path)
    generated_path = os.path.join(GENERATED_CUSTOM_ROOT, generated_path)
    
    ground_audio, _ = librosa.load(ground_path, sr=rate, mono=True)
    generated_audio, _ = librosa.load(generated_path, sr=rate, mono=True)
    min_length = min(len(generated_audio), len(ground_audio))
    generated_audio = generated_audio[:min_length]
    ground_audio = ground_audio[:min_length]

    lsds.append(eval_example_target(generated_audio, ground_audio)['lsd'])
    content_pitches.append(eval_example_content(generated, ground_audio)['pitch_jaccard'])
    content_chromas.append(eval_example_content_chroma(generated_audio, ground_audio)['pitch_chroma_mse'])

print(f"LSD: {sum(lsds) / len(lsds)}")
print(f"Pitch jaccard: {sum(content_pitches) / len(content_pitches)}")
print(f"Pitch chroma MSE: {sum(content_chromas) / len(content_chromas)}")

### Calculate original model with frozen pretrained style encoder (`model-original-pretrained-frozen-25-10-2023`)

In [ ]:
import os

rate = 16_000
HOME_ROOT = ""
GENERATED_CUSTOM_ROOT = "/home/user/ss-vq-vae/experiments/outputs/model-original-pretrained-frozen-25-10-2023/"
with open("outputs/model-original-pretrained-frozen-25-10-2023/vqvae_list") as f:
    generated_custom_audios_list = f.read().splitlines()

lsds = []
content_pitches = []
content_chromas = []
for (_, (_, _, ground_path)), generated_path in tqdm(zip(ground_audios_df.iterrows(), generated_custom_audios_list)):
    ground_path = os.path.join(HOME_ROOT, ground_path)
    generated_path = os.path.join(GENERATED_CUSTOM_ROOT, generated_path)
    
    ground_audio, _ = librosa.load(ground_path, sr=rate, mono=True)
    generated_audio, _ = librosa.load(generated_path, sr=rate, mono=True)
    min_length = min(len(generated_audio), len(ground_audio))
    generated_audio = generated_audio[:min_length]
    ground_audio = ground_audio[:min_length]

    lsds.append(eval_example_target(generated_audio, ground_audio)['lsd'])
    content_pitches.append(eval_example_content(generated, ground_audio)['pitch_jaccard'])
    content_chromas.append(eval_example_content_chroma(generated_audio, ground_audio)['pitch_chroma_mse'])

print(f"LSD: {sum(lsds) / len(lsds)}")
print(f"Pitch jaccard: {sum(content_pitches) / len(content_pitches)}")
print(f"Pitch chroma MSE: {sum(content_chromas) / len(content_chromas)}")

### Calculate original model with unfrozen pretrained style encoder (`model-original-pretrained-unfrozen-24-10-2023`)

In [ ]:
import os

rate = 16_000
HOME_ROOT = ""
GENERATED_CUSTOM_ROOT = "/home/user/ss-vq-vae/experiments/outputs/model-original-pretrained-unfrozen-24-10-2023/"
with open("outputs/model-original-pretrained-unfrozen-24-10-2023/vqvae_list") as f:
    generated_custom_audios_list = f.read().splitlines()

lsds = []
content_pitches = []
content_chromas = []
for (_, (_, _, ground_path)), generated_path in tqdm(zip(ground_audios_df.iterrows(), generated_custom_audios_list)):
    ground_path = os.path.join(HOME_ROOT, ground_path)
    generated_path = os.path.join(GENERATED_CUSTOM_ROOT, generated_path)
    
    ground_audio, _ = librosa.load(ground_path, sr=rate, mono=True)
    generated_audio, _ = librosa.load(generated_path, sr=rate, mono=True)
    min_length = min(len(generated_audio), len(ground_audio))
    generated_audio = generated_audio[:min_length]
    ground_audio = ground_audio[:min_length]

    lsds.append(eval_example_target(generated_audio, ground_audio)['lsd'])
    content_pitches.append(eval_example_content(generated, ground_audio)['pitch_jaccard'])
    content_chromas.append(eval_example_content_chroma(generated_audio, ground_audio)['pitch_chroma_mse'])

print(f"LSD: {sum(lsds) / len(lsds)}")
print(f"Pitch jaccard: {sum(content_pitches) / len(content_pitches)}")
print(f"Pitch chroma MSE: {sum(content_chromas) / len(content_chromas)}")

### Calculate Musaicing LSD

In [13]:
with open('outputs/random_audios/musaicing_list') as f:
    musaicing_audios_list = f.read().splitlines()
    
assert len(musaicing_audios_list) == len(ground_audios_df)

musaicing_lsds = []
for (_, (_, _, ground_path)), generated_path in tqdm(zip(ground_audios_df.iterrows(), generated_custom_audios_list)):
    ground_path = os.path.join(HOME_ROOT, ground_path)
    musaicing_path = os.path.join(GENERATED_ROOT, musaicing_path)
    
    ground_audio, _ = librosa.load(ground_path, sr=rate, mono=True)
    musaicing_audio, _ = librosa.load(musaicing_path, sr=rate, mono=True)
    min_length = min(len(musaicing_audio), len(ground_audio))
    musaicing_audio = musaicing_audio[:min_length]
    ground_audio = ground_audio[:min_length]

    musaicing_lsds.append(eval_example_target(musaicing_audio, ground_audio)['lsd'])

print(sum(musaicing_lsds) / len(musaicing_lsds))

15.367256811356247


### Calculate `cp-content` LSD

In [15]:
HOME_ROOT = ""
content_lsds = []
for _, (content_path, _, ground_path) in ground_audios_df.iterrows():
    content_path = os.path.join(HOME_ROOT, content_path)
    ground_path = os.path.join(HOME_ROOT, ground_path)

    content_audio, _ = librosa.load(content_path, sr=rate, mono=True)
    ground_audio, _ = librosa.load(ground_path, sr=rate, mono=True)
    # TODO: why do lengths differ and how should we handle it?
    min_length = min(len(content_audio), len(ground_audio))
    content_audio = content_audio[:min_length]
    ground_audio = ground_audio[:min_length]

    content_lsds.append(eval_example_target(content_audio, ground_audio)['lsd'])

print(sum(content_lsds) / len(content_lsds))

NameError: name 'eval_example_target' is not defined

### Calculate `cp-style` LSD

In [ ]:
HOME_ROOT = ""
style_lsds = []
for _, (_, style_path, ground_path) in ground_audios_df.iterrows():
    style_path = os.path.join(HOME_ROOT, style_path)
    ground_path = os.path.join(HOME_ROOT, ground_path)

    style_audio, _ = librosa.load(style_path, sr=rate, mono=True)
    ground_audio, _ = librosa.load(ground_path, sr=rate, mono=True)
    # TODO: why do lengths differ and how should we handle it?
    min_length = min(len(style_audio), len(ground_audio))
    style_audio = style_audio[:min_length]
    ground_audio = ground_audio[:min_length]

    style_lsds.append(eval_example_target(style_audio, ground_audio)['lsd'])

print(sum(style_lsds) / len(style_lsds))

## Calculate pitch set

In [9]:
import essentia.standard as estd
import pretty_midi

def get_pitches(audio):
    input_sr, sr = rate, 8000  # Need to resample because of EqualLoudness
    audio = estd.Resample(inputSampleRate=input_sr, outputSampleRate=sr)(audio)
    audio = estd.EqualLoudness(sampleRate=sr)(audio)
    rng = np.random.default_rng(seed=(audio > 0).sum())
    audio = rng.normal(loc=audio, scale=1e-4).astype(audio.dtype)  # To prevent Melodia from crashing
    pitches = estd.MultiPitchMelodia(sampleRate=sr)(audio)
    pitches = [[pretty_midi.utilities.hz_to_note_number(p) for p in pl if not np.isclose(0, p)]
               for pl in pitches]
    pitches = [[int(p + 0.5) for p in pl] for pl in pitches]
    return pitches

def eval_example_content(output, reference):
    pitches_output, pitches_reference = get_pitches(output), get_pitches(reference)
    assert len(pitches_output) == len(pitches_reference)
    jaccard = []
    for pl_output, pl_reference in zip(pitches_output, pitches_reference):
        matches = len(set(pl_output) & set(pl_reference))
        total = len(set(pl_output) | set(pl_reference))
        if total == 0:
            jaccard.append(0)
        else:
            jaccard.append(1 - matches / total)
    jaccard = np.mean(jaccard)
    return {'pitch_jaccard': jaccard}

### Calculate pitch jaccard for the model

In [11]:
import os
from tqdm import tqdm

pitches = []
for (_, (_, _, ground_path)), generated_path in tqdm(zip(ground_audios_df.iterrows(), generated_audios_list)):
    ground_path = os.path.join(HOME_ROOT, ground_path)
    generated_path = os.path.join(GENERATED_ROOT, generated_path)
    
    ground_audio, _ = librosa.load(ground_path, sr=rate, mono=True)
    generated_audio, _ = librosa.load(generated_path, sr=rate, mono=True)
    # TODO: why do lengths differ and how should we handle it?
    min_length = min(len(ground_audio), len(generated_audio))
    ground_audio = ground_audio[:min_length]
    generated_audio = generated_audio[:min_length]

    pitches.append(eval_example_content(generated_audio, ground_audio)['pitch_jaccard'])

print(sum(pitches) / len(pitches))

721it [04:32,  2.65it/s]

0.9435879184078675


### Calculate pitch jaccard for `cp-content`

In [14]:
import os
from tqdm import tqdm
import numpy as np

HOME_ROOT = ""

content_pitches = []
for _, (content_path, _, ground_path) in tqdm(ground_audios_df.iterrows()):
    content_path = os.path.join(HOME_ROOT, content_path)
    ground_path = os.path.join(HOME_ROOT, ground_path)

    content_audio, _ = librosa.load(content_path, sr=rate, mono=True)
    ground_audio, _ = librosa.load(ground_path, sr=rate, mono=True)
    # TODO: why do lengths differ and how should we handle it?
    min_length = min(len(content_audio), len(ground_audio))
    content_audio = content_audio[:min_length]
    ground_audio = ground_audio[:min_length]

    content_pitches.append(eval_example_content(content_audio, ground_audio)['pitch_jaccard'])

print(sum(content_pitches) / len(content_pitches))

1624it [07:50,  3.45it/s]

0.5969705502963264


### Calculate pitch jaccard for `cp-style`

In [17]:
HOME_ROOT = ""

style_pitches = []
for _, (_, style_path, ground_path) in tqdm(ground_audios_df.iterrows()):
    style_path = os.path.join(HOME_ROOT, style_path)
    ground_path = os.path.join(HOME_ROOT, ground_path)

    style_audio, _ = librosa.load(style_path, sr=rate, mono=True)
    ground_audio, _ = librosa.load(ground_path, sr=rate, mono=True)
    # TODO: why do lengths differ and how should we handle it?
    min_length = min(len(style_audio), len(ground_audio))
    style_audio = style_audio[:min_length]
    ground_audio = ground_audio[:min_length]

    style_pitches.append(eval_example_content(style_audio, ground_audio)['pitch_jaccard'])

print(sum(style_pitches) / len(style_pitches))

1624it [08:02,  3.37it/s]

0.9365645292661078
